In [20]:
from networks import OdometryData
import numpy as np
import sympy as sy
from numba import njit, jit

In [30]:
x, y, theta, dt, dr1, dr2 = sy.symbols('x, y, theta, dt, dr1, dr2')
lx, ly = sy.symbols('lx, ly')
x_var = [x, y, theta]
l_var = [lx, ly]
h_sym = sy.Array([sy.sqrt((x - lx)**2 + (y - ly)**2),
                                sy.atan2(ly-y, lx-x)-theta])
h_temp = lambda x, l: np.array([ np.sqrt((x[0] - l[0])**2+(x[1] - l[1])**2), np.arctan2(l[1]-x[1], l[0]-x[0])-x[2]])

In [32]:
h = njit( h_temp )

In [3]:
l = np.array([[21, 168.3333, 315.6667, 463, 463, 315.6667, 168.3333, 21, np.nan],
                [0, 0, 0, 0, 292, 292, 292, 292, np.nan]]).T
data = OdometryData("odometry_particle_data.npz")

/home/contagon/anaconda3/envs/dl/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [47]:
z = data.data['z']
p_mu = data.data['p_mu']

In [48]:
landmark = z[:,:,2]
print(np.isnan(landmark).any())
landmark[np.isnan(landmark)] = 9

True


In [49]:
temp = l[landmark[:,0].astype('int')-1]

In [51]:
h_mu = h_temp(p_mu.T, temp.T)

In [52]:
np.isnan(h_mu).any()

True

In [61]:
z = data.data['z']
z[np.isnan(z)] = 9
p_mu = data.data['p_mu']
zbar = []
for i in range(z.shape[1]):
    li = l[z[:,i,2].astype('int')-1]
    zbar.append( h_temp(p_mu.T, li.T).T )
zbar = np.transpose( np.array(zbar), (1,0,2))

In [77]:
v = z[:,:,:2] - zbar

In [78]:
while np.any( v[:,:,1] > np.pi ):
    v[ v[:,:,1] > np.pi ] -= 2*np.pi
while np.any( v[:,:,1] < -np.pi ):
    v[ v[:,:,1] < -np.pi ] += 2*np.pi

In [86]:
np.nanmin( v[:,:,1] )

-3.129189216208264

In [88]:
np.nanmax( z[:,:,2] )

9.0